In [ ]:
module('load', 'pytorch/gpu-cuda-12.1/2.2.0')

In [ ]:
from deep_learning import AlmaSinglePsfDataset, NoNoiseDataset, SingleTelescopeNoiseDataset
from deep_learning import custom_dataloader
from torch.profiler import profile, record_function, ProfilerActivity


In [ ]:
import torch

#torch.set_float32_matmul_precision('high')

In [ ]:
# dataset=AlmaSinglePsfDataset(catalog_dict=catalog_dict,
#                              psf_name="devon_first_advice_psf_3_pix_16_arcsec",
#                              noise_std=0.0,
#                              broadcasting=False)
dataset=NoNoiseDataset(
        catalog_name="bigger_sources_train",
        grid_width_arcsec=6.0,
        grid_pixel_side=224,
        broadcasting=True,
        final_transform=False
)



In [ ]:
dataloader=custom_dataloader(dataset=dataset, batch_size=256, shuffle=False)

In [ ]:
# myiterator=iter(dataloader)
# import torch
# @torch.no_grad()
# def func():
#     #get 80 batches
#     from tqdm import tqdm
#     for i in tqdm(range(10)):
#         first_batch, _=next(myiterator)
        
        
def func_no_decor():
    #get 80 batches
    from tqdm import tqdm
    for i in tqdm(range(3)):
        first_batch, _=next(myiterator)
        
        
        
# func()


In [ ]:
# myiterator=iter(dataloader)
# %prun func_no_decor()

# Profiling one image generation

In [ ]:
import torch
import torch.nn as nn
from torch.profiler import profile, record_function, ProfilerActivity, tensorboard_trace_handler

dataset=NoNoiseDataset(
        catalog_name="resamp_theta3_val",
        grid_width_arcsec=6.0,
        grid_pixel_side=1000,
        broadcasting=True,
        final_transform=False
)

dataloader=custom_dataloader(dataset=dataset, batch_size=50, shuffle=False)

# --- setup (once) ---
myiterator = iter(dataloader)


def one_image_gen():
    # 1) grab a batch
    imgs, lbls = next(myiterator)
    torch.cuda.synchronize(device="cuda:0")

    


activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]

prof = profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(wait=1, warmup=3, active=5, repeat=1),
    on_trace_ready=tensorboard_trace_handler(dir_name='.', use_gzip=False),
    record_shapes=True,
    with_stack=False,
)

prof.__enter__()
for step in range(10):
    with record_function("image_generation"):
        one_image_gen()
    prof.step()
prof.__exit__(None, None, None)

# Profiling one pass

In [ ]:
from deep_learning import ResNet50
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.profiler import profile, record_function, ProfilerActivity, tensorboard_trace_handler


# --- setup (once) ---
myiterator = iter(dataloader)



# 1) create & move your model
model = ResNet50(num_classes=2).to("cuda")

# 2) set TF32 precision (if you want it)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32      = True
torch.set_float32_matmul_precision("high")

model = torch.compile(
    model,
    backend="inductor",
    fullgraph=False,      # or drop this if you don't need full-graph
    dynamic=False        # or False if your input size is fixed
)



model.train()

# ⚠️ instantiate the loss, don’t just grab the class
criterion = nn.CrossEntropyLoss()  
optimizer = AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-4
)

# --- debugged one_pass ---
def one_pass():
    # 1) grab a batch
    imgs, lbls = next(myiterator)
    
    torch.cuda.synchronize(device="cuda:0")

    #print(f"[Data]  imgs: {imgs.shape}, {imgs.dtype}, device={imgs.device}")
    #print(f"[Data]  lbls: {lbls.shape}, {lbls.dtype}, device={lbls.device}")

    #imgs = imgs.to("cuda")
    lbls = lbls.long().to("cuda")
    optimizer.zero_grad()
    out = model(imgs)
    loss = criterion(out, lbls)
    loss.backward()
    total_norm = torch.nn.utils.clip_grad_norm_(
        model.parameters(),
        max_norm=20.0
    )
    optimizer.step()
    torch.cuda.synchronize(device="cuda:0")



def some_steps(steps=4):
    for i in range(passes):
        one_pass()
        
#%prun some_steps()
# run it once


myiterator=iter(dataloader)
activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]



prof = profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(wait=1, warmup=3, active=5, repeat=1),
    on_trace_ready=tensorboard_trace_handler(dir_name='.', use_gzip=False),
    record_shapes=True,
    with_stack=False,
)

myiterator = iter(dataloader)
prof.__enter__()
for step in range(10):
    with record_function("model_inference"):
        one_pass()
    prof.step()
prof.__exit__(None, None, None)

In [ ]:
from hta.trace_analysis import TraceAnalysis

# Map rank 0 → your single file
trace_files = { 0: "trace.json" }

# Initialize with that mapping
analyzer = TraceAnalysis(trace_files=trace_files, trace_dir=".")

# Now all the usual calls will work:
time_spent_df = analyzer.get_temporal_breakdown()
idle_df, idle_stats_df = analyzer.get_idle_time_breakdown(show_idle_interval_stats=True)
kernel_type_df, kernel_metrics_df = analyzer.get_gpu_kernel_breakdown()

# And if you want the built-in visualizations:
analyzer.get_temporal_breakdown(visualize=True)
analyzer.get_gpu_kernel_breakdown(visualize=True)


In [ ]:
dataloader=custom_dataloader(dataset=dataset, batch_size=1024, shuffle=False)

In [ ]:
myiterator=iter(dataloader)
# import torch
# @torch.no_grad()
# def func():
#     #get 80 batches
#     from tqdm import tqdm
#     for i in tqdm(range(10)):
#         first_batch, _=next(myiterator)
        
        
def func_no_decor():
    #get 80 batches
    from tqdm import tqdm
    for i in tqdm(range(100)):
        first_batch, _=next(myiterator)
        
        
        
# func()


In [ ]:
myiterator_batched=iter(dataloader)

activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]


# with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
#     with record_function("batch_loading"):
#         for i in range(100):
#             first_batch_batched, _=next(myiterator_batched)

            
with profile(activities=activities) as prof:
    for i in range(1):
        first_batch_batched, _=next(myiterator_batched)

prof.export_chrome_trace("trace.json")

In [ ]:
import torch
import matplotlib.pyplot as plt

# Grab one batch from the DataLoader
myiterator  = iter(dataloader)
first_batch, _ = next(myiterator)           # first_batch → [B, C, H, W] (or [B, 1, H, W])

# Make sure we’re in floating-point and on CPU
imgs = first_batch.float().cpu()            # shape: (B, C, H, W)
B     = imgs.size(0)

# Flatten each image to 1-D so reductions are per-image
flat  = imgs.view(B, -1)                    # shape: (B, C*H*W)

# Per-image statistics
max_per_img   = flat.max(dim=1).values.numpy()   # brightest pixel
min_per_img   = flat.min(dim=1).values.numpy()   # faintest pixel
mean_per_img  = flat.mean(dim=1).numpy()         # mean intensity

# ─────────── Plot the three histograms ───────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(max_per_img,  bins=30)
axes[0].set_title('Brightest pixel per image')
axes[0].set_xlabel('Max pixel value')
axes[0].set_ylabel('Count')

axes[1].hist(min_per_img,  bins=30)
axes[1].set_title('Faintest pixel per image')
axes[1].set_xlabel('Min pixel value')

axes[2].hist(mean_per_img, bins=30)
axes[2].set_title('Mean intensity per image')
axes[2].set_xlabel('Mean pixel value')

plt.tight_layout()
plt.show()

In [ ]:
from deep_learning import ResNet50, VisionTransformer
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.profiler import profile, record_function, ProfilerActivity, tensorboard_trace_handler


# --- setup (once) ---
myiterator = iter(dataloader)



# 1) create & move your model
model = VisionTransformer(
    
).to("cuda")

# 2) set TF32 precision (if you want it)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32      = True
torch.set_float32_matmul_precision("high")

model = torch.compile(
    model,
    backend="inductor",
    fullgraph=False,      # or drop this if you don't need full-graph
    dynamic=False        # or False if your input size is fixed
)



model.train()

# ⚠️ instantiate the loss, don’t just grab the class
criterion = nn.CrossEntropyLoss()  
optimizer = AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-4
)

# --- debugged one_pass ---
def one_pass():
    # 1) grab a batch
    imgs, lbls = next(myiterator)
    
    torch.cuda.synchronize(device="cuda:0")

    #print(f"[Data]  imgs: {imgs.shape}, {imgs.dtype}, device={imgs.device}")
    #print(f"[Data]  lbls: {lbls.shape}, {lbls.dtype}, device={lbls.device}")

    #imgs = imgs.to("cuda")
    lbls = lbls.long().to("cuda")
    optimizer.zero_grad()
    out = model(imgs)
    loss = criterion(out, lbls)
    loss.backward()
    total_norm = torch.nn.utils.clip_grad_norm_(
        model.parameters(),
        max_norm=20.0
    )
    optimizer.step()
    torch.cuda.synchronize(device="cuda:0")



def some_steps(steps=4):
    for i in range(passes):
        one_pass()
        
#%prun some_steps()
# run it once


myiterator=iter(dataloader)
activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]



prof = profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(wait=1, warmup=3, active=5, repeat=1),
    on_trace_ready=tensorboard_trace_handler(dir_name='.', use_gzip=False),
    record_shapes=True,
    with_stack=False,
)

myiterator = iter(dataloader)
prof.__enter__()
for step in range(10):
    with record_function("model_inference"):
        one_pass()
    prof.step()
prof.__exit__(None, None, None)